# Thanh's world map figure
This doesnt work in this env. 


In [ ]:
library(tidyverse)
library(readxl)
library(sf)
library(viridis)
library(cowplot)
library(ggrepel)
# library(patchwork)
library(rnaturalearth)
library(rnaturalearthdata)
library(rnaturalearthhires)

record <- read_csv("data/imported.csv")

world <- ne_countries(scale = "medium", returnclass = "sf") %>% 
  filter(name != "Antarctica") %>% 
  st_cast(.,"POLYGON") %>%
  mutate(id = 1:nrow(.))


world_large <- ne_countries(scale = "large", returnclass = "sf") %>% 
  filter(name != "Antarctica")

world_uk <- world %>% 
  left_join(., record, by = c("name" = "country")) %>% 
  st_transform(., crs = "+proj=laea +lat_0=40 +lon_0=10 +x_0=4321000 +y_0=3210000 +ellps=GRS80 +units=m +no_defs")
 
world_uk_cases <- world_uk %>% 
  mutate(count = cut(.$cases, breaks = c(0,5,20,80,100,200,500,600,700,800,1000), labels = c("≤ 5", "6 - 20", "21-80","81-100","101-200","201-500","501-600","601-700","701-800","≥ 800"))) %>% 
  filter(!is.na(cases), id != 443) %>% 
  filter(!(id %in% c(940,941,942,943,944,945,946,948,949,950,951))) %>% 
  group_by(name, count) %>% 
  summarise(geometry = st_union(geometry)) %>% 
  ungroup() %>% 
  mutate(lon=map_dbl(geometry, ~st_centroid(.x)[[1]]),
         lat=map_dbl(geometry, ~st_centroid(.x)[[2]]))

# join_record <- left_join(record, world, by = c("country" = "sovereignt")) %>% 
#   select(country, name_long, cases)


# world_plot <- 
ggplot() +
  geom_sf(data = world_uk, fill= "grey70", size = 0.25) +
  geom_sf(data = world_uk %>% filter(name == "United Kingdom"), fill= "red", size = 0.25) +
  geom_sf(data = world_uk_cases, aes(fill = count), color = "white", size = 0.2, show.legend = T) +
  scale_fill_viridis(na.value='grey70', discrete = T, option="F", name = "") +
  geom_label_repel(data = world_uk_cases, 
                  aes(x=lon, y=lat, label = name), 
                  nudge_x = .1, nudge_y = 0.1, 
                  # point.padding = unit(0.1, "lines"),
                  size = 3.5,
                  force_pull = 5,
                  segment.alpha = 0.3, 
                  min.segment.length = 0.5,
                  alpha = 0.7, show.legend = F, 
                  box.padding = 0.5,
                  segment.curvature = -0.01,
                  segment.ncp = 1,
                  segment.angle = 30,
                  max.overlaps = 50,
                  bg.color = "white", # shadow color
                  bg.r = 0.05,          # shadow radius
                  force = 5) +
  theme_minimal_grid() +
  theme(axis.title.x=element_blank(),
        axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.y=element_blank(),
        legend.position = "right")
  
  # theme_void()
  # theme(panel.background = element_rect(fill = "aliceblue"))

bbox <- st_bbox(c(xmin = -26, ymin = 24.249974, ymax = 74, xmax = 55), crs = st_crs(4326))

bound <- st_as_sfc(bbox)

uk_eu <- st_crop(world, bbox) %>% 
  left_join(., record, by = c("name" = "country")) %>% 
  mutate(count = cut(.$cases, breaks = c(0,5,10,20,80,100,200,500,1000), labels = c("≤ 5", "6 - 10", "11-20","21-80","81-100","101-200","201-500","≥ 500"))) %>% 
  filter(name != "Greenland") %>% 
  group_by(name, count) %>% 
  summarise(geometry = st_union(geometry)) %>% 
  ungroup() %>% 
  mutate(lon=map_dbl(geometry, ~st_centroid(.x)[[1]]),
         lat=map_dbl(geometry, ~st_centroid(.x)[[2]]))
  

eu_plot <- ggplot() +
  geom_sf(data = bound, fill= "aliceblue", size = 0.5, color = "black") +
  geom_sf(data = uk_eu, fill= "grey70", size = 0.25, color = "grey40") +
  geom_sf(data = uk_eu %>% filter(name == "United Kingdom"), fill= "red", size = 0.25, color = "grey40") +
  geom_sf(data = uk_eu %>% filter(!is.na(count)), aes(fill = count), color = "white", size = 0.15, show.legend = T) +
  geom_label_repel(data = uk_eu %>% filter(!is.na(count)), 
                   aes(x=lon, y=lat, label = name), 
                   nudge_x = 0, 
                   nudge_y = 0, 
                   segment.alpha = 0.1, 
                   alpha = 0.8, 
                   fontface = 1,
                   point.padding = NA) +
  scale_fill_viridis(na.value='grey70', discrete = T, option="F", name = "") +
  # theme_minimal_grid()
  theme_void() +
  theme(legend.position = "none")

ggdraw(xlim = c(0,1)) +
  draw_plot(world_plot) +
  draw_plot(eu_plot, x = 0, y = 0.5, height = 0.5, width = 0.5)

